In [ ]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

## 🌱 DALL-E

- 웹사이트: [DALL-E](https://labs.openai.com/)
- 웹사이트에서 이미지를 업로드 하거나, 이미지를 생성하는 기능 구현이 잘 되어 있습니다.
- in-painting, out-painting 도구도 함께 제공합니다.


## Client 생성

- `client` 는 OpenAI 모듈로 생성된 인스턴스 입니다.

[주의] 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.


설치된 openai 버전 확인


In [ ]:
import openai

print(f"설치된 버전: {openai.__version__}")

In [ ]:
from openai import OpenAI

client = OpenAI()

## 1. 이미지 생성(image.generate)

- Reference: https://platform.openai.com/docs/guides/images/introduction?context=node
- `model`
  - 생성: 사용 가능한 모델은 `dall-e-3` 및 `dall-e-2`
  - 편집(edit): `dall-e-2`
  - 기준 이미지에 대한 다양화(variation): `dall-e-2`

**이미지 크기**

- `dall-e-2`
  - `256x256`, `512x512`, `1024x1024`
- `dall-e-3`
  - `1024x1024`, `1792x1024`, `1024x1792`

기본적으로 이미지는 표준 화질로 생성되지만, DALL-E 3을 사용할 때는 화질을 설정할 수 있습니다. "HD"로 설정하면 디테일이 향상됩니다. 정사각형의 표준 화질 이미지가 가장 빠르게 생성됩니다.

**사진요청 개수**

- DALL-E 3을 사용하면 **한 번에 1개의 이미지를 요청** 할 수 있고(병렬 요청을 통해 더 많은 이미지를 요청할 수 있음)
- DALL-E 2를 사용하면 n 매개변수와 함께 **한 번에 최대 10개의 이미지를 요청** 할 수 있습니다.


- 이미지 프롬프트는 이제현 님이 가장 많이 쓴 것 같습니다. 작가상도 받았어요. 
- 참고: https://jehyunlee.github.io/2024/03/20/General-46-dalle3styles3/
- 참고: https://news.kbs.co.kr/news/pc/view/view.do?ncd=7961598


In [ ]:
korean_prompt = "한여름 바닷가 카페의 장면입니다. 한 사람이 책상 없이 리랙스한 소파에 앉아 아이스 아메리카노를 마시며 시원한 바다 경치를 즐기고 있습니다. 카페 내부는 아늑하고 스타일리시하며, 현대적인 디자인을 가지고 있습니다. 큰 유리창이 장면을 지배하며, 맑은 파란 하늘과 넓은 바다, 잔잔한 파도가 보입니다. 초점은 아름다운 바다 경치에 맞춰져 있습니다. 그 사람은 시원한 음료를 음미하며 편안하고 만족스러운 모습입니다."
image_prompt = "A scene at a seaside cafe in midsummer. A person is sitting on a relaxed sofa without a desk, enjoying a refreshing view of the sea while drinking an iced Americano. The cafe interior is cozy and stylish, with a modern design. The large window dominates the scene, showing a clear blue sky and expansive sea with gentle waves. The focus is on the beautiful ocean view. The person is relaxed and content, savoring the cool drink"

In [ ]:
# DALL-E 3 모델을 사용하여 이미지를 생성합니다.
response = client.images.generate(
    model="dall-e-3",
    prompt=image_prompt,
    size="1024x1024",
    quality="standard",
    n=1,
)

# 생성된 이미지의 URL을 저장합니다.
image_url = response.data[0].url

다운로드 받을 수 있는 이미지의 URL을 출력합니다.


In [ ]:
# 생성된 이미지의 URL을 출력합니다.
print(image_url)

Jupyter Notebook 에서 출력하기 위하여 Image 를 사용합니다.


In [ ]:
from IPython.display import Image

# 생성된 이미지를 출력합니다.
Image(url=image_url, width=500)

`urllib` 라이브러리를 사용하여 이미지를 다운로드합니다.


In [ ]:
import urllib

# 생성된 이미지를 URL로부터 다운로드하여 저장합니다.
urllib.request.urlretrieve(image_url, "generated_image.jpg")

## 2. 이미지 수정(image.edit)

- 도큐먼트: https://platform.openai.com/docs/api-reference/images/createEdit

**주요 파라미터**

- `mask`:
  - 마스킹 영역은 완전히 투명한 영역(예: 알파가 0인 곳)이 이미지를 편집할 위치를 나타내는 추가 이미지입니다.
  - 4MB 미만의 유효한 PNG 파일이어야 하며 이미지와 크기가 같아야 합니다.
  - 마스킹한 영역에 대하여 다양한 버전의 이미지를 생성합니다.
- `model`: 이미지 생성에 사용할 모델입니다. 현재 `dall-e-2` 만 지원됩니다.
- `n`: 생성할 이미지의 개수입니다.
- `size`: `256x256`, `512x512`, or `1024x1024`. 기본값은 `1024x1024` 입니다.


In [ ]:
from openai import OpenAI
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO

def display_images(original_path, mask_path):
    # OpenAI 클라이언트 초기화
    client = OpenAI()

    # 원본 이미지와 마스크 이미지 열기
    original_img = Image.open(original_path)
    mask_img = Image.open(mask_path)

    # DALL-E 2 API를 사용하여 이미지 편집
    
    response = client.images.edit(
        model="dall-e-2",
        image=open(original_path, "rb"),
        mask=open(mask_path, "rb"),
        prompt="Teletubbies doll for the masked section.",
        n=1,
        size="1024x1024",
    )
    edited_url = response.data[0].url

    # 편집된 이미지 다운로드
    response = requests.get(edited_url)
    edited_img = Image.open(BytesIO(response.content))

    # 이미지 표시
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    
    axs[0].imshow(original_img)
    axs[0].set_title('Original Image')
    axs[0].axis('off')
    
    axs[1].imshow(mask_img)
    axs[1].set_title('Mask Image')
    axs[1].axis('off')
    
    axs[2].imshow(edited_img)
    axs[2].set_title('Edited Image')
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.show()


# 사용 예시
original_path = "data/sample.png"
mask_path = "data/sample-mask.png"

display_images(original_path, mask_path)

## 3. 이미지 변형(image.create_variation)

주어진 이미지의 변형을 생성합니다.

- 도큐먼트: https://platform.openai.com/docs/api-reference/images/createVariation

주요 파라미터

- `image`: 변형의 기준으로 사용할 이미지입니다. 4MB 미만의 유효한 `PNG` 파일이어야 하며 `정사각형` 이어야 합니다.
- `model`: 이미지 생성에 사용할 모델입니다. 현재 `dall-e-2` 만 지원됩니다.
- `n`: 생성할 이미지의 개수입니다.
- `size`: `256x256`, `512x512`, or `1024x1024`. 기본값은 `1024x1024` 입니다.


In [ ]:
def image_variation(original_path):
    # OpenAI 클라이언트 초기화
    client = OpenAI()

    # 변형 횟수와 이미지 크기 설정
    number_of_variations = 2
    image_size = "512x512"

    # 원본 이미지 열기
    original_img = Image.open(original_path)

    # DALL-E 2 API를 사용하여 이미지 변형 생성
    response = client.images.create_variation(
        image=open(original_path, "rb"),
        n=number_of_variations,
        size=image_size,
    )

    # 변형된 이미지 URL 가져오기
    variation_urls = [data.url for data in response.data]

    # 이미지 표시를 위한 준비
    fig, axs = plt.subplots(1, number_of_variations + 1, figsize=(15, 5))

    # 원본 이미지 표시
    axs[0].imshow(original_img)
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    # 변형된 이미지들 표시
    for i, url in enumerate(variation_urls, start=1):
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        axs[i].imshow(img)
        axs[i].set_title(f'Variation {i}')
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()

# 사용 예시
original_path = "data/doll.png"
image_variation(original_path)